In [ ]:
from pathlib import Path
import rasterio
import geopandas as gpd
import data as dt
import csv
import numpy as np

scene_list = [Path(s) for s in ["GL085796E28265N.tif", "GL085802E28275N.tif", "GL085806E28270N.tif"]]
in_dir = Path("data/raw")
out_dir = Path("data/processed")
y = gpd.read_file(in_dir / "GL_3basins_2015.shp")

fields = ["scene"] + sum([[f"{s}_{i}" for i in range(3)] for s in ["mean", "sdev"]], [])
stat_path = out_dir / "statistics.csv"
f = open(stat_path, "a")
writer = csv.writer(f)
writer.writerow(fields)

for scene in scene_list:
    img = rasterio.open(in_dir / scene)
    result = dt.preprocessor(img, y)
    np.save(out_dir / f"{scene.stem}-labels.npy", result[2])
    writer.writerow([str(scene)] + list(np.hstack(result[:2])))

In [ ]:
import matplotlib.pyplot as plt
for s in result[2]:
    plt.imshow(s)
    plt.show()